In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import os

import tarfile

In [3]:
tf = tarfile.open("instacart_online_grocery_shopping_2017_05_01.tar.gz")

In [4]:
tf.extractall()

In [5]:
os.listdir('instacart_2017_05_01')

['._aisles.csv',
 '._departments.csv',
 '._orders.csv',
 '._order_products__prior.csv',
 '._order_products__train.csv',
 '._products.csv',
 'aisles.csv',
 'departments.csv',
 'orders.csv',
 'order_products__prior.csv',
 'order_products__train.csv',
 'products.csv']

In [6]:
aisles = pd.read_csv('instacart_2017_05_01/aisles.csv')

In [7]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [8]:
department = pd.read_csv('instacart_2017_05_01/departments.csv')

In [9]:
department

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [10]:
orders = pd.read_csv('instacart_2017_05_01/orders.csv')

In [11]:
orders.head(5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [12]:
order_p = orders[orders.eval_set == 'prior']

In [13]:
order_p.shape

(3214874, 7)

In [16]:
orders_train = pd.read_csv('instacart_2017_05_01/order_products__train.csv')

In [17]:
orders_train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [18]:
orders_train.shape

(1384617, 4)

In [19]:
len(orders_train.order_id.unique())

131209

In [20]:
products = pd.read_csv('instacart_2017_05_01/products.csv')

In [21]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [110]:
products = pd.merge(products, department, how='inner', on='department_id')

In [111]:
products.head()

,product_id,product_name,aisle_id,department_id,department
0,1,Chocolate Sandwich Cookies,61,19,snacks
1,16,Mint Chocolate Flavored Syrup,103,19,snacks
2,25,Salted Caramel Lean Protein & Fiber Bar,3,19,snacks
3,32,Nacho Cheese White Bean Chips,107,19,snacks
4,41,Organic Sourdough Einkorn Crackers Rosemary,78,19,snacks


In [112]:
data = pd.merge(orders_train, products,how='inner' ,on='product_id')

In [113]:
data.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department
0,1,49302,1,1,Bulgarian Yogurt,120,16,dairy eggs
1,816049,49302,7,1,Bulgarian Yogurt,120,16,dairy eggs
2,1242203,49302,1,1,Bulgarian Yogurt,120,16,dairy eggs
3,1383349,49302,11,1,Bulgarian Yogurt,120,16,dairy eggs
4,1787378,49302,8,0,Bulgarian Yogurt,120,16,dairy eggs


In [114]:
data = data.sort_values(['order_id'])

In [115]:
data.head(10)

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department
0,1,49302,1,1,Bulgarian Yogurt,120,16,dairy eggs
3627,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,canned goods
3651,1,13176,6,0,Bag of Organic Bananas,24,4,produce
19131,1,47209,7,0,Organic Hass Avocado,24,4,produce
26424,1,22035,8,1,Organic Whole String Cheese,21,16,dairy eggs
1214,1,49683,4,0,Cucumber Kirby,83,4,produce
8,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs
152,1,10246,3,0,Organic Celery Hearts,83,4,produce
36714,36,46979,8,1,Asparagus,83,4,produce
28417,36,39612,1,0,Grated Pecorino Romano Cheese,2,16,dairy eggs


In [116]:
basket = {}
cart_num = 1
cart = []
for i in data.index:
    if(cart_num == data.order_id[i]):
        cart.append(data.department[i]) 
    else:
        basket[cart_num] = cart
        cart = []
        cart_num = data.order_id[i]
        cart.append(data.department[i])

In [117]:
transactions = []

In [118]:
for i in basket.keys():
    transactions.append(tuple(basket[i]))

In [119]:
for i in transactions:
    sorted(i)

In [120]:
transactions[0:10]

[('dairy eggs',
  'canned goods',
  'produce',
  'produce',
  'dairy eggs',
  'produce',
  'dairy eggs',
  'produce'),
 ('produce',
  'dairy eggs',
  'beverages',
  'dairy eggs',
  'produce',
  'dairy eggs',
  'deli',
  'produce'),
 ('produce',
  'produce',
  'produce',
  'frozen',
  'produce',
  'pantry',
  'produce',
  'produce',
  'snacks'),
 ('produce', 'produce', 'frozen', 'produce', 'deli', 'produce', 'frozen'),
 ('produce',
  'beverages',
  'beverages',
  'frozen',
  'beverages',
  'deli',
  'produce',
  'canned goods',
  'canned goods',
  'dairy eggs',
  'dairy eggs',
  'dairy eggs',
  'canned goods',
  'pantry',
  'canned goods',
  'dairy eggs',
  'canned goods',
  'frozen',
  'household',
  'produce',
  'dairy eggs',
  'produce',
  'dairy eggs',
  'produce',
  'household',
  'pantry',
  'produce',
  'dairy eggs',
  'canned goods',
  'canned goods',
  'beverages',
  'dairy eggs',
  'snacks',
  'dairy eggs',
  'frozen',
  'pantry',
  'dairy eggs',
  'deli',
  'canned goods',
  

In [121]:
from efficient_apriori import apriori

In [184]:
itemsets, rules = apriori(transactions, min_support= 0.3)

In [185]:
print(rules)

[{beverages} -> {dairy eggs}, {beverages} -> {produce}, {frozen} -> {dairy eggs}, {produce} -> {dairy eggs}, {dairy eggs} -> {produce}, {snacks} -> {dairy eggs}, {frozen} -> {produce}, {snacks} -> {produce}]


In [186]:
for i in rules:
    print(i)

{beverages} -> {dairy eggs} (conf: 0.699, supp: 0.328, lift: 1.049, conv: 1.109)
{beverages} -> {produce} (conf: 0.730, supp: 0.342, lift: 0.988, conv: 0.968)
{frozen} -> {dairy eggs} (conf: 0.772, supp: 0.300, lift: 1.158, conv: 1.461)
{produce} -> {dairy eggs} (conf: 0.736, supp: 0.544, lift: 1.105, conv: 1.266)
{dairy eggs} -> {produce} (conf: 0.816, supp: 0.544, lift: 1.105, conv: 1.423)
{snacks} -> {dairy eggs} (conf: 0.743, supp: 0.324, lift: 1.115, conv: 1.297)
{frozen} -> {produce} (conf: 0.808, supp: 0.314, lift: 1.094, conv: 1.361)
{snacks} -> {produce} (conf: 0.775, supp: 0.338, lift: 1.049, conv: 1.159)
